In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'  # autodl 将模型的缓存保存到数据盘
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'  # 使用 Hugging Face 镜像站

# 使用领域（私有）数据微调 ChatGLM3

生成带有 epoch 和 timestamp 的模型文件

In [2]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINET

In [3]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
# train_data_path = 'data/zhouyi_dataset_handmade.csv'    # 训练数据路径
train_data_path = 'data/zhouyi_dataset_train.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = 'data/zhouyi_dataset_val.csv'                     # 验证数据路径，如果没有则设置为None
seed = 72                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 4                             # LoRA秩
lora_alpha = 8                            # LoRA alpha值
lora_dropout = 0.1                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

## 数据处理

In [4]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={'train': train_data_path, 'eval': eval_data_path})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 1024
    })
    eval: Dataset({
        features: ['content', 'summary'],
        num_rows: 256
    })
})


In [5]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"], num_examples=8)

,content,summary
0,如何根据需卦的精神，来平衡现代生活的快节奏和内心的平和？,根据需卦的精神，来平衡现代生活的快节奏和内心的平和，我们需要培养一种顺应自然、不强求的态度，合理安排时间，留出空间给自己去等待和思考。
1,讼卦的智慧主要告诫我们什么？,讼卦的智慧告诫我们应谨慎行事，避免不必要的争执，深知稳重和自制的重要性。
2,师卦中的坎水在解读时代表什么？,坎水在师卦中代表困难和挑战，同时也象征军队的养育与智慧。
3,乾卦中“盛极必衰”的警告是什么意思？,“盛极必衰”的警告意味着在成功的高峰期要警惕可能出现的逆转，保持谦逊和警惕。
4,屯卦整体传达的是吉利还是凶险的寓意？,屯卦整体传达的是吉利的寓意，表明虽然初期有困难，但最终能够解决问题。
5,在需卦中，“云登天上而未雨”象征着什么？,这象征着一个事物正在积聚力量，但尚未到达爆发点，提示人们应该等待。
6,在解读比卦时，应该注意什么样的行为准则和人际关系选择？,解读比卦时，我们应该注意保持善良、忠诚，并明智地选择合作伙伴和朋友，强调合作与和谐的重要性，并适宜地做出快速决策。
7,屯卦所代表的主要含义是什么？,屯卦代表起始的艰难与险阻，象征着动而遇险的情形。


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

In [8]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}


In [9]:
column_names = dataset['train'].column_names
tokenized_dataset_train = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)
column_names = dataset['eval'].column_names
tokenized_dataset_eval = dataset['eval'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False,
    remove_columns=column_names
)

In [10]:
tokenized_dataset_train = tokenized_dataset_train.shuffle(seed=seed)
tokenized_dataset_eval = tokenized_dataset_eval.shuffle(seed=seed)
tokenized_dataset_train = tokenized_dataset_train.flatten_indices()
tokenized_dataset_eval = tokenized_dataset_eval.flatten_indices()

In [11]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}


In [12]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

## 加载模型

In [13]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['fp16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True,
                                  revision='b098244')

model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [14]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [15]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [16]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 974,848 || all params: 6,244,558,848 || trainable%: 0.0156


### QLoRA 微调模型

In [17]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 6
output_dir = f"models/{model_name_or_path}-epoch{train_epochs}-{timestamp}"

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                             # 输出目录
    per_device_train_batch_size=32,                    # 每个设备的训练批量大小
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=5e-3,                                # 学习率
    weight_decay=1e-1,                                 # 衰减率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="cosine",                        # 学习率调度器类型
    warmup_ratio=0.2,                                  # 预热比例
    logging_steps=16,                                  # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    eval_strategy="steps",                             # 模型评估策略
    save_steps=16,                                     # 模型保存步数
    save_total_limit=5,                                # 模型保存次数限制
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                         # 是否使用混合精度训练
    load_best_model_at_end=True
)

In [19]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset_train,
        eval_dataset=tokenized_dataset_eval,
        data_collator=data_collator
    )

### 模型训练

In [20]:
trainer.train()

Step,Training Loss,Validation Loss
16,3.709800,3.135545
32,2.664500,2.923963
48,2.169900,2.891411
64,2.035100,2.896318
80,1.669400,2.926096
96,1.644300,2.908519
112,1.324000,2.905502
128,1.242300,2.899324
144,0.921800,3.016282
160,0.896000,3.012658


TrainOutput(global_step=192, training_loss=1.6329020410776138, metrics={'train_runtime': 1166.4118, 'train_samples_per_second': 5.267, 'train_steps_per_second': 0.165, 'total_flos': 5.171725114579354e+16, 'train_loss': 1.6329020410776138, 'epoch': 6.0})

In [21]:
trainer.model.save_pretrained(output_dir)

In [22]:
import torch
torch.cuda.empty_cache()

In [23]:
qlora_model.requires_grad_(False)
model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=4096, out_features=4608, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=4096, out_features=4, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=4, out_features=4608, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (c

In [24]:
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')
base_model.requires_grad_(False)
base_model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [25]:
def compare_chatglm_results(query, model, qlora_model):
    inputs = tokenizer(query, return_tensors="pt").to(0)
    base_out = model.generate(**inputs, max_new_tokens=512)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    
    base_response = tokenizer.decode(base_out[0], skip_special_tokens=True)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后：\n{ft_response}")
    return base_response, ft_response

In [26]:
base_response, ft_response = compare_chatglm_results("乾卦的意义是什么？", base_model, qlora_model)

问题：乾卦的意义是什么？

原始输出：
[gMASK]sop 乾卦的意义是什么？
乾卦是八卦之一，代表天、阳气、强健、积极向上等象征。在中国传统文化中，乾卦具有很高的地位和象征意义。乾卦的意义主要体现在以下几个方面：

1. 代表天：乾卦是八卦中代表天的一个象征，意味着宇宙、天地、自然等。乾卦所代表的天是刚强、积极、充满生机的，它反映了宇宙间万物的生长和发展。

2. 代表阳气：乾卦象征着太阳、火等阳性的能量。在自然界中，阳光是生命之源，它代表着活力、热情、积极向上的精神。乾卦所代表的阳气是推动事物生长的动力，也是人类生存和发展的重要能源。

3. 代表强健：乾卦所代表的强健是充满力量、坚毅不摧的象征。它体现了人们面对困难和挑战时，要有坚定的信念和勇往直前的精神，只有这样才能够战胜困难，取得成功。

4. 代表积极向上：乾卦象征着积极、进取、向上向善的品质。它鼓励人们要不断追求进步，努力向前，克服困难，实现自己的目标和理想。

总之，乾卦在中国传统文化中具有非常重要的象征意义，它代表了天、阳气、强健、积极向上等美好品质和价值观念。


微调后：
[gMASK]sop 乾卦的意义是什么？ 乾卦在周易中代表天，由六阳爻组成，象征纯阳刚健。这一卦象提醒人们，在行事上应保持刚健和积极进取的态度，但也要注意防止过于刚直，以免招致损失。在事业上，乾卦代表刚健和果敢，但也要注意避免独断和刚毅过甚。在婚恋方面，强调阳刚健的重要性，但也要注意避免过于刚强。在决策上，乾卦鼓励人们保持刚健和果敢，但也要注意避免过于刚直和独断。


In [27]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model)

问题：周易中的讼卦是什么

原始输出：
[gMASK]sop 周易中的讼卦是什么？
周易中的讼卦是八卦之一，它的卦象是由两个离字叠加而成，表示诉讼、争端、诉讼等概念。讼卦的意义是解决纷争、和解矛盾，强调通过沟通、协商、妥协等和平方式来解决争端，而不是通过武力、诉讼等强制性手段。

讼卦的卦辞描述了诉讼的过程和结果，其中“初”表示诉讼的开始， “终”表示诉讼的结束， “损”表示通过损失、牺牲来解决问题，“益”表示通过协商、妥协来解决问题。

周易中的讼卦也反映了社会的现实情况，人们在使用诉讼解决争端时，应该遵循公平、公正、公开的原则，尽量避免使用武力、威胁等不道德手段。


微调后：
[gMASK]sop 周易中的讼卦是什么意义 讼卦在周易中代表争执和诉讼，其由下坎上乾组成，象征天与水相背离，象征争执和矛盾。这一卦象提醒我们，在开始行动之前，需要谨慎对待可能出现的争执和风险。象辞提醒我们，在讼卦中，应当避免争讼，而应寻求和谐。在事业上，可能存在反复和困难，但通过合作和耐心，可以克服困难，实现和谐。在决策上，应当谨慎行事，避免争执，而寻求公正。在婚恋上，需要克服争执，以真诚和耐心来解决问题。


In [28]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model)

问题：师卦是什么？

原始输出：
[gMASK]sop 师卦是什么？
师卦是《易经》中的一个卦象，由两个阴爻夹一个阳爻构成，象征着军队或力量。师卦的含义包括：军队、力量、领导、教导等。在《易经》中，师卦通常被认为是一种阳刚之卦，具有积极向上、勇敢担当的品质。同时，师卦也提醒人们要善于教导、引导他人，以达到和谐团结的目的。


微调后：
[gMASK]sop 师卦是什么？ 师卦在周易中代表军队的卦象，由坎（水）上坎（水）下组成，象征军队的团结与和谐。此卦提示在困难中寻求突破和解决困境时，应保持冷静和耐心，以避免盲目行动。师卦鼓励人们面对困境时，要如同军队一样，保持团结和坚韧，以克服困难。在事业上，师卦提醒人们要克服困难，积极应对挑战，以实现目标。在决策中，师卦强调要谨慎行事，避免决策失误。在人际关系中，师卦鼓励人们以耐心和智慧来建立和谐关系。
